In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time

def get_open_interest(symbol="SOLUSDT", interval_minutes=60, days=5*365):
    endpoint = "https://fapi.binance.com/futures/data/openInterestHist"
    start_time = datetime.utcnow() - timedelta(days=days)
    start_ts = int(start_time.timestamp() * 1000)

    all_data = []
    while start_ts < int(datetime.utcnow().timestamp() * 1000):
        params = {
            "symbol": symbol,
            "period": f"{interval_minutes}m",  # 1h interval
            "limit": 500,
            "startTime": start_ts,
        }
        response = requests.get(endpoint, params=params)
        data = response.json()

        if not data or "openInterest" not in data[0]:
            break

        all_data.extend(data)

        # Update start time for next batch
        last_time = int(data[-1]['timestamp'])
        start_ts = last_time + 1

        time.sleep(0.5)  # rate limiting

    # Build DataFrame
    df = pd.DataFrame(all_data)
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df['openInterest'] = pd.to_numeric(df['openInterest'], errors='coerce')

    # Resample to daily by taking the daily average or close
    df.set_index('timestamp', inplace=True)
    df_daily = df['openInterest'].resample('1D').last().dropna()
    df_daily.name = 'daily_open_interest'

    return df_daily.to_frame()

# Example usage
df_oi_sol = get_open_interest()
print(df_oi_sol.head())

/Users/hamzamuhammad/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


KeyError: 0